In [1]:
import polars as pl

In [18]:
data1 = pl.read_excel(source='data/data1.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data2 = pl.read_excel(source='data/data2.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data3 = pl.read_excel(source='data/data3.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data4 = pl.read_excel(source='data/data4.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data5 = pl.read_excel(source='data/data5.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")

In [19]:
fdata1 = data1.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata2 = data2.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata3 = data3.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata4 = data4.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata5 = data5.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())

In [55]:
data = pl.concat([fdata1, fdata2, fdata3, fdata5])
data = data.with_columns(pl.col("ИНН поставщика").str.to_integer())
data = pl.concat([data, fdata4])
data.shape

(16240, 18)

In [59]:
data = data.filter((pl.col("Статус допуска")!=pl.lit("Неизвестно")) & pl.col("Снижение на торгах,%").is_not_null())
data.shape

(14765, 18)